<a href="https://colab.research.google.com/github/RyanXJu/web-Scraper/blob/master/scrape_Realtor_ca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### check if the website allows scraping
#### adding /robots.txt to the original domain

### if the site doesn't allow scraping, we can  define a header to pass along the ***get*** command, which will basically make our queries to the website look like they are coming from an actual browser

## Scrape web

In [5]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [14]:
# define the url to use
# sapo = "https://casa.sapo.pt/Venda/Apartamentos/?sa=11&or=10"

realtor = "https://www.realtor.ca/qc/montreal/real-estate"

response = get(realtor, headers=headers)

In [15]:
print(response)
# list of HTTP status codes : https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

<Response [200]>


In [16]:
print(response.text[:1000])


<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head><script src="/e8469ba99880590089b6cb783b503a90" async></script><meta charset="utf-8" /><meta name="theme-color" content="#D0191D" /><meta name="viewport" content="width=device-width" /><meta name="google" content="notranslate" /><meta http-equiv="CACHE-CONTROL" content="NO-CACHE" /><meta http-equiv="PRAGMA" content="NO-CACHE" /><meta name="GOOGLEBOT" content="NOARCHIVE" /><meta name="format-detection" content="telephone=no" /><meta name="shareaholic:site_id" content="a2430a87e639e6cf0f00273171d97103" />
    <link rel="alternate" hreflang="en" href="https://www.realtor.ca/qc/montreal/real-estate" />
    
    <link rel="alternate" hreflang="fr" href="https://www.realtor.ca/qc/montréal/immobilier" />
    
    <link rel="canonical" href="https://www.realtor.ca/qc/montreal/real-estate" />
    <title>
	Real Estate in Montréal | REALTOR.ca
</title>

    <!-- FavIcon and browser tile defaults (begin)-->



## make the Soup!

In [17]:
html_soup = BeautifulSoup(response.text, 'html.parser')

an object that can be iterated while we scrape the results in each search page






In [71]:
house_containers = html_soup.find_all('div', class_="listingCard card")

In [72]:
first = house_containers[0]
first.find_all('div')

[<div class="listingCardHeader">
 <div class="listingCardHeaderLeft">
 </div>
 <div class="listingCardHeaderRight" data-binding="visible=ShowHeaderIcons">
 <a aria-label="Add Notes" class="propertyCardDetailsNoteIcon noteIcon listingCardActionIcon" data-binding="data-value={Id}_{AddressForNotes}_{Latitude}_{Longitude}" data-value="21600898_24 Rue Roger, Montréal (L'Île-Bizard/Sainte-Geneviève), Quebec _45.478_-73.925661" href="#" style="text-decoration: none; display: none;">
 <span class="listingCardNoteIcon m_al fa fa-sticky-note" style="font-size: 31px; color: #F3E500;"></span></a>
 <a aria-label="Add to My Favourites" class="propertyCardDetailsFavouriteIcon favouriteIcon listingCardActionIcon" data-binding="data-value={Id}_{Latitude}_{Longitude}_{AppMode}" data-value="21600898_45.478_-73.925661_1" href="#" onclick="" style="text-decoration: none;">
 <img alt="" src="/images/common/icons/svg/heart-empty.svg" style="height: 27px; width: 31px;"/>
 </a>
 </div>
 </div>, <div class="lis

In [78]:
# price
price = first.find_all("div", class_="listingCardPrice")[0].text

import re
price = re.sub("[^\d\.]", "", price)
print(price)

# address
address = first.find_all("div", class_="listingCardAddress")[0].text.lstrip()
print(address)

# img url
img = str(first.find_all("img")[1])
img
imgurl = img[img.find("src=", img.find("src")+1)+4 : len(img)-8] # find the second occurance of "src="
imgurl

49000
24 Rue Roger, Montréal (L'Île-Bizard/Sainte-Geneviève), Quebec 
                    




In [76]:
# get all the links
for url in first.find_all('a', class_="blockLink listingDetailsLink" ):
    print(url.get('href'))
    # adding the domain part to the url
    url_complet = "https://www.realtor.ca"+ url.get("href")
    print(url_complet)


/real-estate/21600898/24-rue-roger-montréal-lîle-bizardsainte-geneviève-lîle-bizard
https://www.realtor.ca/real-estate/21600898/24-rue-roger-montréal-lîle-bizardsainte-geneviève-lîle-bizard


In [137]:
img = str(first.find_all("img")[1])
img


'<img alt="24 Rue Roger, Montréal (L\'Île-Bizard/Sainte-Geneviève), Quebec " class="listingCardImage" data-binding="src=ImageURL,alt=Address" src="https://cdn.realtor.ca/listings/TS637177241770070000/reb5/medres/7/22124917_1.jpg">\n</img>'

In [144]:
imgurl = img[img.find("src=", img.find("src")+1)+4 : len(img)-8] # find the second occurance of "src="
imgurl

'"https://cdn.realtor.ca/listings/TS637177241770070000/reb5/medres/7/22124917_1.jpg"'